# Explore TPM data matrix

In [1]:
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from Bio import SeqIO

from override import GENE_NAME_OVERRIDE, GENE_PRODUCT_OVERRIDE
import get_top_gene_set as gtgs



## Load expression data

In [2]:
# load TPM data
data_file = 'data/extract_TPM_counts.tsv'
sample2condition_file = 'data/sample2condition.txt'
sample_file = None #'config/samples_to_include.txt'
condition_file = 'config/conditions_to_include.txt'

df, sample2condition, samples, conditions = gtgs.load_data(data_file, sample2condition_file, sample_file, condition_file)


In [3]:
df.head()

,locus_tag,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,gene_len,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,EQU24_RS00005,chromosomal replication initiator protein DnaA,CDS,dnaA,NZ_CP035467.1,0,1317,Derived by automated computational analysis us...,MSALWNNCLAKLENEISSSEFSTWIRPLQAIETDGQIKLLAPNRFV...,1318,...,38.557373,38.810668,37.444214,40.246006,40.100118,33.432274,39.880174,38.355431,30.247582,41.248441
1,EQU24_RS00010,DNA polymerase III subunit beta,CDS,,NZ_CP035467.1,1502,2603,Derived by automated computational analysis us...,MKYIINREQLLVPLQQIVSVIEKRQTMPILSNVLMVFRENTLVMTG...,1102,...,52.552767,52.461746,42.676553,49.210083,46.798476,48.142385,45.465136,46.498139,37.152951,52.902410
2,EQU24_RS00015,DNA replication/repair protein RecF,CDS,recF,NZ_CP035467.1,3060,4140,Derived by automated computational analysis us...,MSLQKLDIFNVRNIRQASLQPSPGLNLIYGANASGKSSVLEAIFIL...,1081,...,31.350991,34.914128,21.479309,24.204682,22.171104,22.006566,22.658157,22.753325,19.407103,29.834124
3,EQU24_RS00020,DNA topoisomerase (ATP-hydrolyzing) subunit B,CDS,gyrB,NZ_CP035467.1,4185,6600,Derived by automated computational analysis us...,MSENIKQYDSTNIQVLKGLDAVRKRPGMYIGDTDDGTGLHHMVFEV...,2416,...,74.848501,80.850761,54.959319,64.911376,59.653059,64.648318,69.119079,65.643179,57.590223,68.306759
4,EQU24_RS00025,hypothetical protein,CDS,,NZ_CP035467.1,6825,7062,Derived by automated computational analysis us...,VKTTKYFLTTRMRPDREIIKDEWIQYVVRFPENEHIQFDGRIRRWA...,238,...,50.324948,49.349547,34.539657,36.521074,37.789611,39.358066,38.992158,35.870964,41.462392,40.227192


## Load features from genbank

In [4]:
# feature tuple indices
LEFT_IDX = 0
RIGHT_IDX = 1
STRAND_IDX = 2
LOCUS_IDX = 3
GENE_IDX = 4
TYPE_IDX = 5

# Load features from genbank
gb_file = "data/5GB1c_sequence.gb"
pos_feats, neg_feats = gtgs.get_pos_neg_features(gb_file)

In [5]:
def get_override_gene(locus_tag,cur_gene):
    '''
    Given a locus tag, return an overridden gene
    '''
    return GENE_NAME_OVERRIDE[locus_tag] if locus_tag in GENE_NAME_OVERRIDE else cur_gene

def insert_override_gene_names(feat_list):
    '''
    Given a list of features as a tuple, update some of the 
    locus info with gene names
    '''
    #convert tuple to list
    feat_list = [list(x) for x in feat_list]
    
    updated_feats = []
    for feat in feat_list:
        g = get_override_gene(feat[LOCUS_IDX],feat[GENE_IDX])
        feat[GENE_IDX] = g
        updated_feats.append(feat)
        
    return updated_feats
    

In [6]:
# update some gene names with a set of names to override (helpful for viz)
pos_feats = insert_override_gene_names(pos_feats)
neg_feats = insert_override_gene_names(neg_feats)

print(pos_feats[:5])
print(neg_feats[:5])

[[0, 1317, 1, 'EQU24_RS00005', 'dnaA', 'CDS'], [1502, 2603, 1, 'EQU24_RS00010', '', 'CDS'], [3060, 4140, 1, 'EQU24_RS00015', 'recF', 'CDS'], [4185, 6600, 1, 'EQU24_RS00020', 'gyrB', 'CDS'], [6825, 7062, 1, 'EQU24_RS00025', '', 'CDS']]
[[20389, 21388, -1, 'EQU24_RS00080', '', 'CDS'], [23251, 23782, -1, 'EQU24_RS00095', '', 'CDS'], [23841, 24321, -1, 'EQU24_RS00100', '', 'CDS'], [24340, 24994, -1, 'EQU24_RS00105', 'crp', 'CDS'], [32310, 32679, -1, 'EQU24_RS00130', '', 'CDS']]


## Determine loci which may be in operons

In [7]:
op_min_dist = 120
maybe_operon_loci = gtgs.flag_potential_operon_loci(pos_feats, neg_feats, op_min_dist)

print(f"Number of possible operon loci at distance of {op_min_dist}bp: {len(maybe_operon_loci)}/{len(pos_feats)+len(neg_feats)}")

Number of possible operon loci at distance of 120bp: 2068/4431


## Transform the data frame for mean TPM calculations

In [8]:
# for the loaded TPM matrix, which column contains the unique gene ids?
LOCUS_ID_COL = 'locus_tag'
# use this column to get a full list of all genes for which expression was measured
LOCI = list(df[LOCUS_ID_COL].values)

In [14]:
df_means = gtgs.get_average_tpm_by_condition(df,samples,conditions,sample2condition,LOCI)
df_means

locus_tag,exp_condition,EQU24_RS00005,EQU24_RS00010,EQU24_RS00015,EQU24_RS00020,EQU24_RS00025,EQU24_RS00030,EQU24_RS00035,EQU24_RS00040,EQU24_RS00045,...,EQU24_RS22110,EQU24_RS22115,EQU24_RS22120,EQU24_RS22125,EQU24_RS22130,EQU24_RS22135,EQU24_RS22140,EQU24_RS22145,EQU24_RS22150,EQU24_RS22155
0,MeOH,23.333155,18.915775,18.453916,18.267805,16.960643,12.377795,43.815536,9.670950,7.302145,...,1298.257682,15.624619,20.208066,26.004364,20.960234,28.719983,93.616437,161.528124,496.990651,280.344047
1,NO3_lowO2_slow_growth,32.050358,43.656760,21.351623,62.267687,41.684925,31.921455,57.849768,16.885694,14.926147,...,6497.868109,26.273485,28.945133,23.525245,26.432667,35.167264,178.996199,164.083806,433.438735,493.895115
2,NoCu,44.348687,59.629360,28.268717,56.818319,49.839406,38.394652,81.530362,40.501969,36.576500,...,8345.785345,43.065124,34.380565,44.419579,34.601933,65.339879,253.608495,273.284694,731.052190,1087.621126
3,NoLanthanum,33.444023,43.689839,23.172675,57.297047,42.367072,41.941657,102.513601,30.226787,19.462312,...,5085.637409,16.423284,35.588138,44.623117,43.201743,21.927260,109.783330,67.277718,211.575175,328.943746
4,WithLanthanum,35.462185,41.792237,20.644554,57.130166,34.258335,46.201637,110.721781,31.813805,19.438086,...,3942.957792,15.972203,34.318829,49.216725,40.000662,21.220809,98.100610,73.116973,194.389586,319.998959
5,highCu,47.861477,79.109490,33.534043,73.330408,48.662214,33.986359,92.999818,51.950784,50.370579,...,8132.547467,48.894308,35.608730,46.109300,30.125207,89.710150,342.981435,386.493127,1021.453762,1692.401154
6,highO2_slow_growth,64.784508,99.002970,44.856281,78.997757,77.842263,56.626268,97.721756,35.735531,28.808125,...,3468.582202,40.548782,48.532405,37.139500,38.204218,52.033315,220.196691,244.139008,505.427610,561.847119
7,lowCH4,30.829331,33.532522,18.491160,42.963648,31.643505,24.308334,74.948663,18.601900,12.935098,...,7477.339715,18.356915,24.390308,20.231568,19.721043,60.397912,226.909132,297.029289,874.637567,506.211825
8,lowCu,42.973556,61.209155,28.828713,61.573321,50.966799,31.319574,75.047593,40.038670,35.331019,...,7157.344557,43.386082,33.574108,36.872718,31.899782,66.743497,293.599291,313.741841,843.607251,1123.669681
9,lowO2_fast_growth,35.736190,43.159066,27.325800,48.976066,33.463183,26.694205,72.505275,15.430765,12.595934,...,9584.028559,26.951394,26.808972,31.889009,23.882390,72.207950,334.033049,376.513322,1178.068431,1183.180057


In [12]:
# get top locus ids in all conditions
n = 3
top_locs = gtgs.get_top_n_perc_by_condition(df_means, LOCI, n)

# filter out loci maybe in operons
top_locs_op_filter_out = [x for x in top_locs if x in maybe_operon_loci]
print(f'Top {n}%: {len(top_locs)} ({len(top_locs) - len(top_locs_op_filter_out)} filtered)\n')


Top 3%: 37 (25 filtered)



# Visualize tradeoffs in top N% threshold used

In [16]:
tradeoff_data = []

# examine top sets using a range of top 1% to top 10%
for i in range(1,11):
    # get top locus ids in all conditions
    top_locs = gtgs.get_top_n_perc_by_condition(df_means, LOCI, i)
    top_locs_op_filter_out = [x for x in top_locs if x in maybe_operon_loci]
    
    # get the average expression of each locus across conditions
    means = []
    for loc in top_locs:
        mean_exp = np.mean(df_means[loc].values)
        means.append(mean_exp)
        
    # add row to the data frame consisting of:
    # n: top % threshold
    # locus count: number of top loci identified
    # locus count filtered: locus count after filtering operon genes
    # min expression: the minimum avg expression value from the top gene set
    row = [i,
           len(top_locs),
           len(top_locs)-len(top_locs_op_filter_out),
           min(means)
          ]
    
    tradeoff_data.append(row)
    
tradeoff_df = pd.DataFrame(tradeoff_data, columns=['n','loc_count','loc_count_filt','min_exp'])


In [17]:
tradeoff_df

,n,loc_count,loc_count_filt,min_exp
0,1,14,8,3022.573901
1,2,26,17,1653.480931
2,3,37,25,1131.648318
3,4,56,35,930.223684
4,5,84,43,667.493775
5,6,107,55,469.300182
6,7,127,65,369.788442
7,8,153,78,313.001223
8,9,178,92,313.001223
9,10,197,103,259.999672


In [75]:
num_genes_layer = alt.Chart(tradeoff_df).mark_circle(size=100).encode(
    x=alt.X('loc_count:Q', axis=alt.Axis(title='Number of loci in top set')),
    y=alt.Y('min_exp:Q', axis=alt.Axis(title='Minimum ave. TPM of loci in top set')),
    #x='loc_count:Q',
    #y='min_exp:Q',
    color=alt.Color('n:O', scale=alt.Scale(scheme='viridis'),legend=None),
    tooltip=[alt.Tooltip('n:Q', title='Top N%'),
             alt.Tooltip('loc_count:Q', title='Num loci in top set'),
             alt.Tooltip('loc_count_filt:Q', title='Num loci in top set after operon filtering'),
             alt.Tooltip('min_exp:Q', title='Min. ave. expr. in top set')
            ]
)

op_ex_layer = alt.Chart(tradeoff_df).mark_point(size=100).encode(
    x='loc_count_filt:Q',
    y='min_exp:Q',
    color=alt.Color('n:O', scale=alt.Scale(scheme='viridis'),legend=None),
    tooltip=[alt.Tooltip('n:Q', title='Top N%'),
             alt.Tooltip('loc_count:Q', title='Num loci in top set'),
             alt.Tooltip('loc_count_filt:Q', title='Num loci in top set after operon filtering'),
             alt.Tooltip('min_exp:Q', title='Min. ave. expr. in top set')
             ]
)

rule = alt.Chart(tradeoff_df).mark_rule().encode(
    x='loc_count_filt:Q',
    x2='loc_count:Q',
    y='min_exp:Q',
    color=alt.Color('n:O', scale=alt.Scale(scheme='viridis'),legend=None),
)

text = alt.Chart(tradeoff_df).mark_text(
    align='left',
    baseline='middle',
    angle=335,
    dx=5,
    dy=-7,
    size=14,
    color='black'
).encode(
    x='loc_count:Q',
    y='min_exp:Q',
    text=alt.Text('n:N'),
    #color=alt.Color('n:O', scale=alt.Scale(scheme='viridis'),legend=None),
).transform_calculate(n='"Top " + datum.n + "%"')

# combine layers into chart
chart = num_genes_layer + op_ex_layer + rule + text

# final chart configs
chart.properties(
    title='Tradeoff between top gene set % threshold'
).configure_title(
    fontSize=20,
).configure_axis(
    labelFontSize=14,
    titleFontSize=16
)

alt.LayerChart(...)

In [15]:
def load_override_dict(filename):
    with open(filename, 'r') as f:
        lines = [x.strip().split('\t') for x in f.readlines()]
        override_dict = dict(lines)
        
    return override_dict

gene_ov = load_override_dict("data/gene_name_override.txt")
prod_ov = load_override_dict("data/gene_product_override.txt")